# **Implementing KNN from scratch**

The test problem we will be using in this tutorial is the iris classification.

The problem consists of 150 observations of iris flowers from three different species. There are 4 measurements of given flowers: sepal length, sepal width, petal length, and petal width (all in the same unit of centimeters). The predicted attribute is the species, which is either Setosa, Versicolor, or Virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct (typically 96% or better).

**By**: LOUBAR Ahcene

# **Loading and preparing data**

-- For this KNN mini project we will use the Iris dataset that is available on the sklearn library and also on kaggle and other sources.

In [ ]:
import csv

with open('/content/iris.data.txt', 'r') as csvfile:
  lines = csv.reader(csvfile)
  for row in lines :
    print (', '.join(row))

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [ ]:
import random

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
  with open(filename, 'r') as csvfile:
    lines = csv.reader(csvfile)
    dataset = list(lines)
    for x in range(len(dataset)-1):
      for y in range(4):
        dataset[x][y] = float(dataset[x][y])
      if random.random() < split:
        trainingSet.append(dataset[x])
      else:
        testSet.append(dataset[x])

#We can test this function out with our iris dataset, as follows:
trainingSet=[]
testSet=[]

loadDataset('/content/iris.data.txt', 0.66, trainingSet, testSet)

print ('Train: ' + repr(len(trainingSet)))
print ('Test: ' + repr(len(testSet)) )


Train: 99
Test: 50


In [ ]:
trainingSet[:5]

[[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
 [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
 [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
 [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
 [5.4, 3.9, 1.7, 0.4, 'Iris-setosa']]

As we can see our training set has 4 features and a label column.

# **KNN Sub-functions**

To compute the distance we can use three possible distance formulas: Euclidean, Manhattan and Winkowski.

**--Euclidean distane**:


In [ ]:
import math

def euclideanDistance(instance1, instance2, length):
        return math.sqrt(sum(list(map(lambda x,y: math.pow(x-y,2),instance1[:length],instance2[:length]))))
#Example
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']

distance = euclideanDistance(data1, data2, 3)
print ('Euclidean Distance: ' + repr(distance))

Euclidean Distance: 3.4641016151377544


**--Manhattan distance**:

In [ ]:
def manhattanDistance(instance1, instance2, length):
        return sum(list(map(lambda x, y: abs(x-y), instance1[:length], instance2[:length])))

**--Minkowski distance**:
Let's note that the Minkowski formula is a generalized version of the Euclidean and the Manhattan one: we have a parameter p in the Minkowski formula, if we put p = 1 we'll get the Manhattan distance and if we put p = 2, we'll get the Euclidean distance. For the Minkowski distance we'll take the middle: p = 3/2

In [ ]:
def minkowskiDistance(instance1, instance2, length):
        return math.pow(sum(list(map(lambda x, y: math.pow(abs(x-y),(3/2)), instance1[:length], instance2[:length]))), 2/3)

We have a fuction that computes the distance between our new instance and all the training observations and returns the K nearest neighbors of it.

In [ ]:
import operator

def getNeighbors(trainingSet, testInstance, k, distance):

        distances = []

        length = len(testInstance)-1

        for x in range(len(trainingSet)):
                if(distance == "Euclidean"):
                        dist = euclideanDistance(testInstance, trainingSet[x], length)
                elif(distance == "Manhattan"):
                        dist = manhattanDistance(testInstance, trainingSet[x], length)
                elif(distance == "Minkowski"):
                        dist = minkowskiDistance(testInstance, trainingSet[x], length)

                distances.append((trainingSet[x], dist))
        # we sort the distances
        distances.sort(key=operator.itemgetter(1))

        neighbors = []

        for x in range(k):
                #we take the k nearest neighbors, so the instances with the shortest distance
                neighbors.append(distances[x][0])

        return neighbors


#Example
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]

testInstance = [5, 5, 5]

k = 1

neighbors = getNeighbors(trainSet, testInstance, k, "Euclidean")

print(neighbors)

[[4, 4, 4, 'b']]


Now we have to write a function that will return the most common class in our list of k nearest neighbors.

In [ ]:
import operator

def getResponse(neighbors):
        classVotes = {}

        for x in range(len(neighbors)):

                response = neighbors[x][-1]

                if response in classVotes:
                        classVotes[response] += 1
                else: classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
        #print(sortedVotes)

        return sortedVotes[0][0]

#Example
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

response = getResponse(neighbors)

print(response)

a


In [ ]:
def getAccuracy(testSet, predictions):
        c = 0
        for x in range(len(testSet)):
                if testSet[x][-1] == predictions[x]: c +=1
        return (c/float(len(testSet))) * 100.0

#Example
test = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

preds = ['a', 'a', 'a']

accuracy = getAccuracy(test, preds)

print(accuracy)

66.66666666666666


# **Main KNN**

In [ ]:
def KNN(dataset, instance, k, distance = 'Euclidean'):
        return getResponse(getNeighbors(dataset,instance, k, distance))

In [ ]:
print(getAccuracy(testSet, list(map(lambda x: KNN(trainingSet, x, k=9), testSet))))
print(getAccuracy(testSet, list(map(lambda x: KNN(trainingSet, x, distance="Manhattan", k=9), testSet))))
print(getAccuracy(testSet, list(map(lambda x: KNN(trainingSet, x, distance="Minkowski", k=9), testSet))))

96.0
94.0
96.0
